# Auxiliar Final

In [16]:
import pandas as pd
import numpy as np
import string

In [28]:
def clean_technique(df):
    # Creamos una tabla con todas las palabras disponibles
    aux2 = pd.DataFrame(df.assign(list_aux=lambda df: df.medium_text.apply(str.split)).list_aux.to_list())

    # Pasamos todo a una sola columna que se llame word
    aux3 = pd.DataFrame([np.nan])

    for col in aux2.columns:
        aux3 = pd.concat((aux3, aux2[col]), ignore_index=True)

    aux3.dropna(inplace=True)
    aux3.reset_index(drop=True, inplace=True)
    aux3.rename(columns={0:'word'}, inplace=True)

    # Extraemos las palabras más importantes
    words = (aux3
             .word
             .apply(lambda x: x.translate(str.maketrans('', '', string.punctuation)))
             .apply(lambda x: x.lower())
             .reset_index()
             .groupby('word')
             .size()
             .reset_index()
             .sort_values(0, ascending=False)
             .rename(columns={0:'cuenta'})
             .assign(len_word=lambda x: x.word.str.len())
             .query('len_word > 2')
             .query('cuenta > 100')
             .sort_values('word')
             .word
             .values
            )

    # Volvemos a separar en lista de palabras
    aux = df.assign(list_technique=lambda df: df
                 .medium_text
                 .apply(lambda x: x.translate(str.maketrans('', '', string.punctuation)))
                 .apply(lambda x: x.lower())
                 .apply(lambda x: x.split())
                )

    # Revisamos qué palabras están en la lista que creamos
    for word in words:
        aux = (aux
                 .assign(word_aux=lambda df: df
                         .list_technique
                         .apply(lambda x: word in x)
                         .astype('int8')
                        )
                 .rename(columns={'word_aux':word})
               )

    # Quitamos las columnas que ya no sirven
    aux.drop(columns=['medium_text', 'and', 'with', 'list_technique', 'country', 'source'], inplace=True)
    aux.reset_index(drop=True, inplace=True)
    
    # Cambiamos las opciones de pandas para que se puedan ver todas las palabras
    pd.options.display.max_columns=999
    return aux

In [32]:
datos = pd.read_csv('../datasets/clean/clean_mut_art_ver2_ajustado.csv', index_col=0, parse_dates=['fecha'])
datos.reset_index(drop=True, inplace=True)
datos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10512 entries, 0 to 10511
Data columns (total 14 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   title        10512 non-null  object        
 1   height       10458 non-null  float64       
 2   length       10358 non-null  float64       
 3   art_type     10512 non-null  object        
 4   medium_text  10512 non-null  object        
 5   date_text    10512 non-null  int64         
 6   age          10512 non-null  int64         
 7   house        10512 non-null  object        
 8   fecha        10512 non-null  datetime64[ns]
 9   price        10451 non-null  float64       
 10  author       10512 non-null  object        
 11  performance  10451 non-null  float64       
 12  source       10512 non-null  object        
 13  country      10512 non-null  object        
dtypes: datetime64[ns](1), float64(4), int64(2), object(7)
memory usage: 1.1+ MB


In [52]:
'''
Factores: author, year, month, house, date_text
Regresores: height, length, age, performance, 'acrylic', 'aquatint',
       'arches', 'black', 'board', 'bronze', 'brown', 'canvas', 'cardboard',
       'charcoal', 'color', 'colors', 'colours', 'crayon', 'down', 'etching',
       'gelatin', 'gouache', 'graphite', 'guarro', 'handmade', 'ink', 'laid',
       'lithograph', 'lithographs', 'masonite', 'media', 'metal', 'mixed',
       'mixografía', 'mixograph', 'mounted', 'nylon', 'oil', 'painted',
       'panel', 'paper', 'pastel', 'patina', 'pencil', 'plexiglas',
       'plexiglass', 'print', 'printed', 'rice', 'rods', 'sand', 'screenprint',
       'silkscreen', 'silver', 'tempera', 'watercolor', 'wood', 'wove'


Queremos la interacción de year con mes
'''
datos_fin = clean_technique(datos)
datos_fin['year'] = datos_fin.fecha.dt.year
datos_fin['month'] = datos_fin.fecha.dt.month
datos_fin['ln_precio'] = datos_fin.price.apply(np.log)
datos_fin['ln_height'] = datos_fin.height.apply(np.log)
datos_fin['ln_width'] = datos_fin.length.apply(np.log)
datos_fin['age_2'] = datos_fin.age ** 2
datos_fin['age_3'] = datos_fin.age ** 3
datos_fin['age_4'] = datos_fin.age ** 4
datos_fin.head()

,title,height,length,art_type,date_text,age,house,fecha,price,author,performance,acrylic,aquatint,arches,black,board,bronze,brown,canvas,cardboard,charcoal,color,colors,colours,crayon,down,etching,gelatin,gouache,graphite,guarro,handmade,ink,laid,lithograph,lithographs,masonite,media,metal,mixed,mixografía,mixograph,mounted,nylon,oil,painted,panel,paper,pastel,patina,pencil,plexiglas,plexiglass,print,printed,rice,rods,sand,screenprint,silkscreen,silver,tempera,watercolor,wood,wove,year,month,ln_precio,ln_height,ln_width,age_2,age_3,age_4
0,"""experiencia""",127.00,40.0,charity auction of the salvatorians,2021,1,"dorotheum, vienna",2022-09-21,705.0,jerson jimenez,2.000000,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2022,9,6.558198,4.844187,3.688879,1,1,1
1,relief 8,104.14,41.0,modern and contemporary art,1970,52,"dorotheum, vienna",2022-09-20,5154.0,julio le parc,1.219045,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,2022,9,8.547528,4.645736,3.713572,2704,140608,7311616
2,portrait of a young woman,65.10,50.2,travel & exploration,1952,70,bonhams,2022-09-20,1485.0,raúl anguiano,1.019988,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2022,9,7.303170,4.175925,3.916015,4900,343000,24010000
3,cenizas,142.24,76.0,pop & contemporary art,1974,48,tgp auction,2022-09-20,231.0,jorge camacho,0.600000,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2022,9,5.442418,4.957516,4.330733,2304,110592,5308416
4,panther,264.16,151.0,huge modern art auction - live online,2021,1,ozbid auctions,2022-09-20,549.0,jose haro,0.400000,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2022,9,6.308098,5.576555,5.017280,1,1,1


In [54]:
datos_fin.dropna().to_csv('../datasets/clean/dataset_final.csv', index=False)